In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Input, Add, SeparableConv2D, Resizing
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import precision_score, recall_score, f1_score
import time
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import ResNet50

In [9]:
# Configuración de parámetros
NUM_CLASES = 10
EPOCHS = 10
BATCH_SIZE = 64
IMAGE_SIZE = (224, 224)  # Tamaño requerido por VGG16 y ResNet50

# Cargar el dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Definir el tamaño del subconjunto (ejemplo: 10% de los datos de entrenamiento)
subset_size = int(0.0001 * x_train.shape[0])

# Seleccionar índices aleatorios para el subconjunto
indices = np.random.choice(x_train.shape[0], subset_size, replace=False)

# Crear el subconjunto
x_train_subset = x_train[indices]
y_train_subset = y_train[indices]

# Normalizar los valores de los píxeles en el rango [0, 1]
x_train_resized = x_train.astype('float32') / 255.0
x_test_resized = x_test.astype('float32') / 255.0

# Convertir las etiquetas a one-hot encoding
y_train_cat = to_categorical(y_train, NUM_CLASES)
y_test_cat = to_categorical(y_test, NUM_CLASES)

# Lista para almacenar los resultados de cada modelo
results = []

In [10]:
# 2.5. Función para construir Transfer Learning (ResNet50))
def build_fine_tuning_resnet50(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Congelar todas las capas inicialmente
    for layer in base_model.layers:
        layer.trainable = False

    model = Sequential()
    model.add(Resizing(224, 224, input_shape=input_shape))
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

# 2.6. Función para construir Fine Tuning (ResNet50)
def fine_tune_resnet50(model, base_model, x_train, y_train, x_test, y_test, epochs=10, batch_size=32):
    # Descongelar las últimas 4 capas del modelo base
    for layer in base_model.layers[-4:]:
        layer.trainable = True

    # Compilar el modelo con una tasa de aprendizaje más baja
    optimizer = Adam(learning_rate=1e-5)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    start_time = time.time()
    history = model.fit(
        x_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, y_test),
        verbose=2
    )
    end_time = time.time()

    # Predicciones
    y_pred = np.argmax(model.predict(x_test), axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Cálculo de métricas
    accuracy = np.mean(y_pred == y_true)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    # Almacenar resultados
    results.append({
        'Modelo': 'Fine Tuning ResNet50 (Final)',
        'Exactitud': accuracy,
        'Precisión': precision,
        'Recall': recall,
        'F1-score': f1,
        'Tiempo de Entrenamiento (s)': end_time - start_time
    })

    print(f"Modelo: Fine Tuning ResNet50 (Final)")
    print(f"Exactitud: {accuracy:.4f}, Precisión: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
    print(f"Tiempo de entrenamiento: {end_time - start_time:.2f} segundos\n")

    return history, end_time - start_time

In [11]:
def train_and_evaluate(model, model_name, x_train, y_train, x_test, y_test, epochs=EPOCHS, batch_size=BATCH_SIZE, fine_tune=False):
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    start_time = time.time()
    history = model.fit(
        x_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, y_test),
        verbose=2
    )
    end_time = time.time()

    # Predicciones
    y_pred = np.argmax(model.predict(x_test), axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Cálculo de métricas
    accuracy = np.mean(y_pred == y_true)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    # Almacenar resultados
    results.append({
        'Modelo': model_name,
        'Exactitud': accuracy,
        'Precisión': precision,
        'Recall': recall,
        'F1-score': f1,
        'Tiempo de Entrenamiento (s)': end_time - start_time
    })

    print(f"Modelo: {model_name}")
    print(f"Exactitud: {accuracy:.4f}, Precisión: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
    print(f"Tiempo de entrenamiento: {end_time - start_time:.2f} segundos\n")

    return history, end_time - start_time


In [12]:
# 5.1. Transfer Learning con ResNet50
print("Entrenando Transfer Learning con ResNet50...")
model_resnet50 = build_fine_tuning_resnet50((224, 224, 3), NUM_CLASES)
history_resnet50, time_resnet50 = train_and_evaluate(model_resnet50, 'Fine Tuning ResNet50 (Inicial)', x_train_resized, y_train_cat, x_test_resized, y_test_cat)

# 5.2. Fine Tuning con ResNet50 (Entrenamiento Inicial con Capas Congeladas)
print("Entrenando Fine Tuning con ResNet50 (Inicial)...")
# Extraer el modelo base de ResNet50 para Fine Tuning
base_model_resnet50 = model_resnet50.layers[0]

# Realizar Fine Tuning (Descongelar últimas 4 capas y reentrenar)
history_resnet50_finetune, time_resnet50_finetune = fine_tune_resnet50(
    model_resnet50, 
    base_model_resnet50, 
    x_train_resized, 
    y_train_cat, 
    x_test_resized, 
    y_test_cat
)

Entrenando Transfer Learning con ResNet50...
Epoch 1/10


KeyboardInterrupt: 

In [ ]:
# Crear DataFrame con los resultados
df_results = pd.DataFrame(results)
print(df_results)

# Gráficas de las métricas
metrics = ['Exactitud', 'Precisión', 'Recall', 'F1-score', 'Tiempo de Entrenamiento (s)']
num_metrics = len(metrics)

plt.figure(figsize=(20, 15))

for i, metric in enumerate(metrics, 1):
    plt.subplot(3, 2, i)
    plt.bar(df_results['Modelo'], df_results[metric], color='skyblue')
    plt.title(metric)
    plt.ylabel(metric)
    plt.xticks(rotation=45, ha='right')
    for index, value in enumerate(df_results[metric]):
        if metric != 'Tiempo de Entrenamiento (s)':
            plt.text(index, value + 0.005, f"{value:.2f}", ha='center', va='bottom')
        else:
            plt.text(index, value + 1, f"{value:.2f}", ha='center', va='bottom')

plt.tight_layout()
plt.show()